In [1]:
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
from glob import glob

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
print('pytorch version:', torch.__version__)
global_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', global_device)
from torch.utils.data import DataLoader, random_split, Dataset
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics.classification import MultilabelAveragePrecision

pytorch version: 2.2.0.dev20231027+cu121
Device: cuda:0


In [2]:
# Matplotlib settings
import matplotlib
import matplotlib as mp
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from matplotlib.colors import colorConverter

titlesize = 20
labelsize = 16
legendsize = labelsize
xticksize = 14
yticksize = xticksize

matplotlib.rcParams['legend.markerscale'] = 1.5     # the relative size of legend markers vs. original
matplotlib.rcParams['legend.handletextpad'] = 0.5
matplotlib.rcParams['legend.labelspacing'] = 0.4    # the vertical space between the legend entries in fraction of fontsize
matplotlib.rcParams['legend.borderpad'] = 0.5       # border whitespace in fontsize units
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['axes.labelsize'] = labelsize
matplotlib.rcParams['axes.titlesize'] = titlesize

matplotlib.rc('xtick', labelsize=xticksize)
matplotlib.rc('ytick', labelsize=yticksize)
matplotlib.rc('legend', fontsize=legendsize)

matplotlib.rc('font', **{'family':'serif'})

# Data loading

In [3]:
df_trainval = pd.read_csv('data/train.csv')
df_train, df_val = train_test_split(df_trainval, test_size=0.2, random_state=42)
df_trainval['tags'] = df_trainval['tags'].apply(lambda x: np.array(list(map(int, x.split(',')))))
df_train['tags'] = df_train['tags'].apply(lambda x: np.array(list(map(int, x.split(',')))))
df_val['tags'] = df_val['tags'].apply(lambda x: np.array(list(map(int, x.split(',')))))

df_test = pd.read_csv('data/test.csv')

In [4]:
global_idx2embeds = {} # {idx: np.ndarray[n, 768])}
for npy_file in tqdm(glob('data/track_embeddings/*')):
    track_idx = int(npy_file.split('\\')[1].split('.')[0])
    embeds = np.load(npy_file)
    global_idx2embeds[track_idx] = embeds[:-2]

100%|██████████| 76714/76714 [01:06<00:00, 1147.95it/s]


In [5]:
emb_train_mean = 0
for track_idx in tqdm(df_trainval['track']):
    emb_train_mean += global_idx2embeds[track_idx].mean(axis=0)
emb_train_mean /= len(df_trainval['track'])
emb_train_mean.shape

100%|██████████| 51134/51134 [00:28<00:00, 1809.14it/s]


(768,)

In [6]:
class TrackDataset(torch.utils.data.Dataset):
    def __init__(self, df_tags, test=False):
        self.df_tags = df_tags
        self.test = test

    def __len__(self):
        return len(self.df_tags)

    def __getitem__(self, idx):
        track_idx = self.df_tags.iloc[idx]['track']
        embeds = global_idx2embeds[track_idx]
        if self.test:
            return track_idx, embeds
        labels_onehot = np.zeros(256)
        labels_onehot[self.df_tags.iloc[idx]['tags']] = 1
        return track_idx, embeds, labels_onehot

In [7]:
def collate(batch):
    track_idxs = torch.IntTensor(np.vstack([triplet[0] for triplet in batch]))
    embeds_list = [torch.FloatTensor(triplet[1][:-2] - emb_train_mean).to(global_device) for triplet in batch]
    embeds = pack_sequence(embeds_list, enforce_sorted=False).to(global_device)
    labels_onehot = torch.FloatTensor(np.vstack([triplet[2] for triplet in batch])).to(global_device)
    return track_idxs, embeds, labels_onehot

def collate_test(batch):
    track_idxs = torch.IntTensor(np.vstack([triplet[0] for triplet in batch]))
    embeds_list = [torch.FloatTensor(triplet[1][:-2] - emb_train_mean) for triplet in batch]
    embeds = pack_sequence(embeds_list, enforce_sorted=False).to(global_device)
    return track_idxs, embeds

batch_size = 256

dataset_train = TrackDataset(df_train)
dataloader_train = DataLoader(dataset_train, batch_size, shuffle=True, collate_fn=collate)

dataset_trainval = TrackDataset(df_trainval)
dataloader_trainval = DataLoader(dataset_trainval, batch_size, shuffle=True, collate_fn=collate)

dataset_val = TrackDataset(df_val)
dataloader_val = DataLoader(dataset_val, batch_size, shuffle=False, collate_fn=collate)
dataloader_val_astest = DataLoader(dataset_val, batch_size, shuffle=False, collate_fn=collate_test)

dataset_test = TrackDataset(df_test, test=True)
dataloader_test = DataLoader(dataset_test, batch_size, shuffle=False, collate_fn=collate_test)

In [8]:
class TrackToTags(pl.LightningModule):
    def __init__(self, num_classes=256, input_dim=768, hidden_dim=512, relu_on=True,
                 grulayers=2, gru_in_out=[(768, 768), (768, 768)], poolkernels=[(3,1), (3,1), (3,1)]):
        super().__init__()
        self.num_classes = num_classes
        self.grulayers = grulayers
        self.poolkernels = poolkernels
        self.gru_in_out = gru_in_out
        self.relu_on = relu_on
        gru_list = []
        for i in range(self.grulayers):
            gru_list.append(nn.GRU(gru_in_out[i][0], gru_in_out[i][1], batch_first=True, bidirectional=False, 
                                   num_layers=1).to(global_device))
        
        self.grus = torch.nn.ModuleList(modules=gru_list)
        
        pool_list = []
        for kernel in poolkernels:
            pool_list.append(torch.nn.MaxPool2d(kernel_size=kernel, 
                                                padding=(kernel[0]//2,kernel[1]//2)).to(global_device))
        self.maxpools = torch.nn.ModuleList(modules=pool_list)

        self.bn = nn.LayerNorm(gru_in_out[-1][1])
        self.relu = nn.ReLU()
        self.fc = nn.Linear(gru_in_out[-1][1], num_classes)
        self.loss = nn.BCEWithLogitsLoss()
        
        self.metric = average_precision_score
        self._transit_val = {'preds': [], 'labels': []}
        
    def forward(self, embeds):
        unpacked, lengths = pad_packed_sequence(embeds, batch_first=True)
        x = unpacked
        x = self.maxpools[0](unpacked)
        for layer in range(self.grulayers):
            new_lengths = np.ceil(lengths/self.poolkernels[layer][0])
            x = pack_padded_sequence(x, new_lengths, 
                                     batch_first=True, enforce_sorted=False)
            gruout, h = self.grus[layer](x)
            unpacked, lengths = pad_packed_sequence(gruout, batch_first=True)
            x = self.maxpools[layer+1](unpacked)
                
        x = [(v.sum(0)/length*self.poolkernels[-1][0]).unsqueeze(0) for v, length in zip(x, lengths)]
        x = torch.cat(x, dim = 0)
        x = self.bn(x)
        if self.relu_on:
            x = self.relu(x)
        outs = self.fc(x)
        return outs

    def training_step(self, batch, batch_idx):
        track_idxs, embeds_list, labels_onehot = batch
        pred_logits = self(embeds_list)
        loss = self.loss(pred_logits, labels_onehot)
        self.log("train_loss", loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        track_idxs, embeds_list, labels_onehot = batch
        pred_logits = self(embeds_list)
        loss = self.loss(pred_logits, labels_onehot)
        self.log("val_loss", loss, prog_bar=True)
        
        pred_probs = torch.sigmoid(pred_logits)
        self._transit_val['labels'].append(np.array(labels_onehot.int().cpu()))
        self._transit_val['preds'].append(np.array(pred_probs.cpu()))

    def on_validation_epoch_end(self):
        preds = np.vstack(self._transit_val['preds'])
        labels = np.vstack(self._transit_val['labels'])
        ap = self.metric(labels, preds)
        print(ap)
        self.log('val_ap', ap, prog_bar=True)
        self._transit_val['labels'] = []
        self._transit_val['preds'] = []

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=4e-4)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5)
        
        return {"optimizer": optimizer,
                "lr_scheduler": {
                                "scheduler": scheduler,
                                "interval": "epoch",
                                "frequency": 10
                                },
               }

In [37]:
poolkernels = [(1, 1), (1, 1)]  # (1,1) = no pooling
gru_in_out = [(768, 512)]
layers = (len(gru_in_out))
io_log = '-'.join([f"{io[0]}x{io[1]}" for io in gru_in_out])
kernelslog = '-'.join([f"{kernel[0]}x{kernel[1]}" for kernel in poolkernels])
checkpoint_callback = ModelCheckpoint(dirpath=f'lightning_logs/trv2_grupool_l-{layers}_gruio-{io_log}_krnls-{kernelslog}/',
                                      filename='{epoch}-{val_loss:.3f}-{val_ap:.3f}', 
                                      save_top_k=-1, 
                                      monitor="val_ap", 
                                      every_n_epochs=1)
trainer = pl.Trainer(accelerator="gpu", devices=1, val_check_interval=1.0, 
                     max_epochs=5, log_every_n_steps=100,
                     callbacks=[checkpoint_callback])
# model_1 = TrackToTags(grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)
trainer.fit(model_1, dataloader_trainval, dataloader_val)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params
-----------------------------------------------
0 | grus     | ModuleList        | 2.0 M 
1 | maxpools | ModuleList        | 0     
2 | bn       | LayerNorm         | 1.0 K 
3 | relu     | ReLU              | 0     
4 | fc       | Linear            | 131 K 
5 | loss     | BCEWithLogitsLoss | 0     
-----------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.406     Total estimated model params size (MB)


Epoch 4: 100%|██████████| 200/200 [02:23<00:00,  1.39it/s, v_num=298, train_loss=0.0345, val_loss=0.0361, val_ap=0.386]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 200/200 [02:23<00:00,  1.39it/s, v_num=298, train_loss=0.0345, val_loss=0.0361, val_ap=0.386]


In [10]:
class Baseline(pl.LightningModule):
    def __init__(self, num_classes=256, input_dim=768, hidden_dim=512):
        super().__init__()
        self.num_classes = num_classes
        self.bn = nn.LayerNorm(hidden_dim)
        self.projector =  nn.Linear(input_dim, hidden_dim)
        self.lin = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim)
        )
        self.fc = nn.Linear(hidden_dim, num_classes)

        self.pos_weights = torch.FloatTensor(np.ones(256)).to(global_device)
        self.loss = nn.BCEWithLogitsLoss(pos_weight=self.pos_weights)
        self.metric = average_precision_score
        self._transit_val = {'preds': [], 'labels': []}
        
    def forward(self, embeds_list):
        unpacked, lengths = pad_packed_sequence(embeds_list, batch_first=True)
        x = [self.projector(x) for x in unpacked]
        x = [(v.sum(0)/length).unsqueeze(0) for v, length in zip(x, lengths)]
        x = self.bn(torch.cat(x, dim = 0))
        x = self.lin(x)
        outs = self.fc(x)
        return outs

    def training_step(self, batch, batch_idx):
        track_idxs, embeds_list, labels_onehot = batch
        pred_logits = self(embeds_list)
        loss = self.loss(pred_logits, labels_onehot)
        self.log("train_loss", loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        track_idxs, embeds_list, labels_onehot = batch
        pred_logits = self(embeds_list)
        loss = self.loss(pred_logits, labels_onehot)
        self.log("val_loss", loss, prog_bar=True)
        
        pred_probs = torch.sigmoid(pred_logits)
        self._transit_val['labels'].append(np.array(labels_onehot.int().cpu()))
        self._transit_val['preds'].append(np.array(pred_probs.cpu()))

    def on_validation_epoch_end(self):
        preds = np.vstack(self._transit_val['preds'])
        labels = np.vstack(self._transit_val['labels'])
        ap = self.metric(labels, preds)
        self.log('val_ap', ap, prog_bar=True)
        self._transit_val['labels'] = []
        self._transit_val['preds'] = []

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=3e-4)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5)
        
        return {"optimizer": optimizer,
                "lr_scheduler": {
                                "scheduler": scheduler,
                                "interval": "epoch",
                                "frequency": 10
                                },
               }

In [30]:
checkpoint_callback = ModelCheckpoint(dirpath=f'lightning_logs/baseline/',
                                      filename='{epoch}-{val_loss:.3f}-{val_ap:.3f}', 
                                      save_top_k=-1, 
                                      monitor="val_ap", 
                                      every_n_epochs=1)
trainer = pl.Trainer(accelerator="gpu", devices=1, val_check_interval=1.0, 
                     max_epochs=10, log_every_n_steps=100,
                     callbacks=[checkpoint_callback])
# model_2 = Baseline()
trainer.fit(model_2, dataloader_train, dataloader_val)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | bn        | LayerNorm         | 1.0 K 
1 | projector | Linear            | 393 K 
2 | lin       | Sequential        | 526 K 
3 | fc        | Linear            | 131 K 
4 | loss      | BCEWithLogitsLoss | 0     
------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.210     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 160/160 [01:13<00:00,  2.19it/s, v_num=222, train_loss=0.0421, val_loss=0.047, val_ap=0.216] 

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 160/160 [01:13<00:00,  2.19it/s, v_num=222, train_loss=0.0421, val_loss=0.047, val_ap=0.216]


In [9]:
poolkernels = [(1, 1), (3, 1), (3, 1)]  # (1,1) = no pooling
gru_in_out = [(768, 512), (512, 512)]
layers = (len(gru_in_out))
io_log = '-'.join([f"{io[0]}x{io[1]}" for io in gru_in_out])
kernelslog = '-'.join([f"{kernel[0]}x{kernel[1]}" for kernel in poolkernels])
checkpoint_callback = ModelCheckpoint(dirpath=f'lightning_logs/trv_grupool_l-{layers}_gruio-{io_log}_krnls-{kernelslog}/',
                                      filename='{epoch}-{val_loss:.3f}-{val_ap:.3f}', 
                                      save_top_k=-1, 
                                      monitor="val_ap", 
                                      every_n_epochs=1)
trainer = pl.Trainer(accelerator="gpu", devices=1, val_check_interval=1.0, 
                     max_epochs=23, log_every_n_steps=100,
                     callbacks=[checkpoint_callback])
model_3 = TrackToTags(grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)
trainer.fit(model_3, dataloader_trainval, dataloader_val)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params
-----------------------------------------------
0 | grus     | ModuleList        | 3.5 M 
1 | maxpools | ModuleList        | 0     
2 | bn       | LayerNorm         | 1.0 K 
3 | relu     | ReLU              | 0     
4 | fc       | Linear            | 131 K 
5 | loss     | BCEWithLogitsLoss | 0     
-----------------------------------------------
3.7 M     Trainable params
0         Non-trainable params
3.7 M     Total params
14.710    Total estimated model params size (MB)


Epoch 22: 100%|██████████| 200/200 [03:03<00:00,  1.09it/s, v_num=297, train_loss=0.0342, val_loss=0.0345, val_ap=0.414]

`Trainer.fit` stopped: `max_epochs=23` reached.


Epoch 22: 100%|██████████| 200/200 [03:03<00:00,  1.09it/s, v_num=297, train_loss=0.0342, val_loss=0.0345, val_ap=0.414]


In [50]:
poolkernels = [(1, 1), (2, 1), (2, 1)]  # (1,1) = no pooling
gru_in_out = [(768, 1024), (1024, 512)]
layers = (len(gru_in_out))
io_log = '-'.join([f"{io[0]}x{io[1]}" for io in gru_in_out])
kernelslog = '-'.join([f"{kernel[0]}x{kernel[1]}" for kernel in poolkernels])
checkpoint_callback = ModelCheckpoint(dirpath=f'lightning_logs/grupool_l-{layers}_gruio-{io_log}_krnls-{kernelslog}/',
                                      filename='{epoch}-{val_loss:.3f}-{val_ap:.3f}', 
                                      save_top_k=-1, 
                                      monitor="val_ap", 
                                      every_n_epochs=1)
trainer = pl.Trainer(accelerator="gpu", devices=1, val_check_interval=1.0, 
                     max_epochs=20, log_every_n_steps=100,
                     callbacks=[checkpoint_callback])
model_4 = TrackToTags(grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)
trainer.fit(model_4, dataloader_train, dataloader_val)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params
-----------------------------------------------
0 | grus     | ModuleList        | 7.9 M 
1 | maxpools | ModuleList        | 0     
2 | bn       | LayerNorm         | 1.0 K 
3 | relu     | ReLU              | 0     
4 | fc       | Linear            | 131 K 
5 | loss     | BCEWithLogitsLoss | 0     
-----------------------------------------------
8.0 M     Trainable params
0         Non-trainable params
8.0 M     Total params
32.024    Total estimated model params size (MB)


Epoch 19: 100%|██████████| 160/160 [05:29<00:00,  0.49it/s, v_num=227, train_loss=0.0359, val_loss=0.0455, val_ap=0.231]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 160/160 [05:29<00:00,  0.49it/s, v_num=227, train_loss=0.0359, val_loss=0.0455, val_ap=0.231]


In [52]:
poolkernels = [(2, 1), (2, 1), (2, 1)]  # (1,1) = no pooling
gru_in_out = [(768, 512), (512, 512)]
layers = (len(gru_in_out))
io_log = '-'.join([f"{io[0]}x{io[1]}" for io in gru_in_out])
kernelslog = '-'.join([f"{kernel[0]}x{kernel[1]}" for kernel in poolkernels])
checkpoint_callback = ModelCheckpoint(dirpath=f'lightning_logs/grupool_l-{layers}_gruio-{io_log}_krnls-{kernelslog}/',
                                      filename='{epoch}-{val_loss:.3f}-{val_ap:.3f}', 
                                      save_top_k=-1, 
                                      monitor="val_ap", 
                                      every_n_epochs=1)
trainer = pl.Trainer(accelerator="gpu", devices=1, val_check_interval=1.0, 
                     max_epochs=20, log_every_n_steps=100,
                     callbacks=[checkpoint_callback])
model_5 = TrackToTags(grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)
trainer.fit(model_5, dataloader_train, dataloader_val)
del model_5

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params
-----------------------------------------------
0 | grus     | ModuleList        | 3.5 M 
1 | maxpools | ModuleList        | 0     
2 | bn       | LayerNorm         | 1.0 K 
3 | relu     | ReLU              | 0     
4 | fc       | Linear            | 131 K 
5 | loss     | BCEWithLogitsLoss | 0     
-----------------------------------------------
3.7 M     Trainable params
0         Non-trainable params
3.7 M     Total params
14.710    Total estimated model params size (MB)


Epoch 19: 100%|██████████| 160/160 [01:15<00:00,  2.12it/s, v_num=228, train_loss=0.0375, val_loss=0.0451, val_ap=0.231]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 160/160 [01:15<00:00,  2.11it/s, v_num=228, train_loss=0.0375, val_loss=0.0451, val_ap=0.231]


In [53]:
poolkernels = [(2, 1), (2, 1), (2, 1), (1, 1)]  # (1,1) = no pooling
gru_in_out = [(768, 512), (512, 512), (512, 512)]
layers = (len(gru_in_out))
io_log = '-'.join([f"{io[0]}x{io[1]}" for io in gru_in_out])
kernelslog = '-'.join([f"{kernel[0]}x{kernel[1]}" for kernel in poolkernels])
checkpoint_callback = ModelCheckpoint(dirpath=f'lightning_logs/grupool_l-{layers}_gruio-{io_log}_krnls-{kernelslog}/',
                                      filename='{epoch}-{val_loss:.3f}-{val_ap:.3f}', 
                                      save_top_k=-1, 
                                      monitor="val_ap", 
                                      every_n_epochs=1)
trainer = pl.Trainer(accelerator="gpu", devices=1, val_check_interval=1.0, 
                     max_epochs=20, log_every_n_steps=100,
                     callbacks=[checkpoint_callback])
model_6 = TrackToTags(grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)
trainer.fit(model_6, dataloader_train, dataloader_val)
del model_6

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params
-----------------------------------------------
0 | grus     | ModuleList        | 5.1 M 
1 | maxpools | ModuleList        | 0     
2 | bn       | LayerNorm         | 1.0 K 
3 | relu     | ReLU              | 0     
4 | fc       | Linear            | 131 K 
5 | loss     | BCEWithLogitsLoss | 0     
-----------------------------------------------
5.3 M     Trainable params
0         Non-trainable params
5.3 M     Total params
21.014    Total estimated model params size (MB)


Epoch 19: 100%|██████████| 160/160 [01:21<00:00,  1.95it/s, v_num=229, train_loss=0.037, val_loss=0.0454, val_ap=0.229]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 160/160 [01:22<00:00,  1.95it/s, v_num=229, train_loss=0.037, val_loss=0.0454, val_ap=0.229]


In [54]:
poolkernels = [(1, 1), (2, 1), (2, 1)]  # (1,1) = no pooling
gru_in_out = [(768, 256), (256, 128)]
layers = (len(gru_in_out))
io_log = '-'.join([f"{io[0]}x{io[1]}" for io in gru_in_out])
kernelslog = '-'.join([f"{kernel[0]}x{kernel[1]}" for kernel in poolkernels])
checkpoint_callback = ModelCheckpoint(dirpath=f'lightning_logs/grupool_l-{layers}_gruio-{io_log}_krnls-{kernelslog}/',
                                      filename='{epoch}-{val_loss:.3f}-{val_ap:.3f}', 
                                      save_top_k=-1, 
                                      monitor="val_ap", 
                                      every_n_epochs=1)
trainer = pl.Trainer(accelerator="gpu", devices=1, val_check_interval=1.0, 
                     max_epochs=20, log_every_n_steps=100,
                     callbacks=[checkpoint_callback])
model_7 = TrackToTags(grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)
trainer.fit(model_7, dataloader_train, dataloader_val)
del model_7

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params
-----------------------------------------------
0 | grus     | ModuleList        | 936 K 
1 | maxpools | ModuleList        | 0     
2 | bn       | LayerNorm         | 256   
3 | relu     | ReLU              | 0     
4 | fc       | Linear            | 33.0 K
5 | loss     | BCEWithLogitsLoss | 0     
-----------------------------------------------
969 K     Trainable params
0         Non-trainable params
969 K     Total params
3.878     Total estimated model params size (MB)


Epoch 19: 100%|██████████| 160/160 [01:19<00:00,  2.02it/s, v_num=230, train_loss=0.0479, val_loss=0.0477, val_ap=0.164]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 160/160 [01:19<00:00,  2.02it/s, v_num=230, train_loss=0.0479, val_loss=0.0477, val_ap=0.164]


In [40]:
poolkernels = [(1, 1), (3, 1), (3, 1)]  # (1,1) = no pooling
gru_in_out = [(768, 512), (512, 512)]
layers = (len(gru_in_out))
io_log = '-'.join([f"{io[0]}x{io[1]}" for io in gru_in_out])
kernelslog = '-'.join([f"{kernel[0]}x{kernel[1]}" for kernel in poolkernels])
checkpoint_callback = ModelCheckpoint(dirpath=f'lightning_logs/trv_grupoolaccum_l-{layers}_gruio-{io_log}_krnls-{kernelslog}/',
                                      filename='{epoch}-{val_loss:.3f}-{val_ap:.3f}', 
                                      save_top_k=-1, 
                                      monitor="val_ap", 
                                      every_n_epochs=1)
trainer = pl.Trainer(accelerator="gpu", devices=1, val_check_interval=1.0, 
                     max_epochs=30, log_every_n_steps=100,
                     accumulate_grad_batches=1024//batch_size,
                     callbacks=[checkpoint_callback]
                     )
model_7 = TrackToTags(grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)
trainer.fit(model_7, dataloader_trainval, dataloader_val)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params
-----------------------------------------------
0 | grus     | ModuleList        | 3.5 M 
1 | maxpools | ModuleList        | 0     
2 | bn       | LayerNorm         | 1.0 K 
3 | relu     | ReLU              | 0     
4 | fc       | Linear            | 131 K 
5 | loss     | BCEWithLogitsLoss | 0     
-----------------------------------------------
3.7 M     Trainable params
0         Non-trainable params
3.7 M     Total params
14.710    Total estimated model params size (MB)


Epoch 29: 100%|██████████| 200/200 [03:20<00:00,  1.00it/s, v_num=299, train_loss=0.0424, val_loss=0.0439, val_ap=0.227]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 200/200 [03:20<00:00,  1.00it/s, v_num=299, train_loss=0.0424, val_loss=0.0439, val_ap=0.227]


In [ ]:
poolkernels = [(1, 1), (3, 1), (3, 1)]  # (1,1) = no pooling
gru_in_out = [(768, 512), (512, 512)]
layers = (len(gru_in_out))
io_log = '-'.join([f"{io[0]}x{io[1]}" for io in gru_in_out])
kernelslog = '-'.join([f"{kernel[0]}x{kernel[1]}" for kernel in poolkernels])
checkpoint_callback = ModelCheckpoint(dirpath=f'lightning_logs/grupoolworelu_l-{layers}_gruio-{io_log}_krnls-{kernelslog}/',
                                      filename='{epoch}-{val_loss:.3f}-{val_ap:.3f}', 
                                      save_top_k=-1, 
                                      monitor="val_ap", 
                                      every_n_epochs=1)
trainer = pl.Trainer(accelerator="gpu", devices=1, val_check_interval=1.0, 
                     max_epochs=23, log_every_n_steps=100,
                     callbacks=[checkpoint_callback]
                     )
model_8 = TrackToTags(grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels, relu_on=False)
trainer.fit(model_8, dataloader_train, dataloader_val)

In [11]:
def predict(model, loader):
    model.to(global_device)
    model.eval()
    track_idxs = []
    predictions = []
    with torch.no_grad():
        for data in loader:
            track_idx, embeds = data
            pred_logits = model(embeds)
            pred_probs = torch.sigmoid(pred_logits)
            predictions.append(pred_probs.cpu().numpy())
            track_idxs.append(track_idx.numpy())
    predictions = np.vstack(predictions)
    track_idxs = np.vstack(track_idxs).ravel()
    return track_idxs, predictions

In [21]:
# for metalearning
poolkernels = [(1, 1), (1, 1)]
gru_in_out = [(768, 512)]
layers = (len(gru_in_out))
model_1 = TrackToTags.load_from_checkpoint('lightning_logs/grupool_l-1_gruio-768x512_krnls-1x1-1x1/epoch=9-val_loss=0.045-val_ap=0.236.ckpt', 
                                           grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)

poolkernels = [(1, 1), (3, 1), (3, 1)]
gru_in_out = [(768, 512), (512, 512)]
layers = (len(gru_in_out))
model_3 = TrackToTags.load_from_checkpoint('lightning_logs/grupool_l-2_gruio-768x512-512x512_krnls-1x1-3x1-3x1/epoch=19-val_loss=0.045-val_ap=0.238.ckpt', 
                                           grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)

model_7 = TrackToTags.load_from_checkpoint('lightning_logs/grupoolaccum2_l-2_gruio-768x512-512x512_krnls-1x1-3x1-3x1/epoch=23-val_loss=0.044-val_ap=0.241.ckpt', 
                                           grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)

# model_8 = TrackToTags.load_from_checkpoint('lightning_logs/grupoolworelu_l-2_gruio-768x512-512x512_krnls-1x1-3x1-3x1/epoch=25-val_loss=0.045-val_ap=0.239.ckpt', 
#                                            grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels, relu_on=False)


In [44]:
poolkernels = [(1, 1), (1, 1)]
gru_in_out = [(768, 512)]
layers = (len(gru_in_out))
model_1 = TrackToTags.load_from_checkpoint('lightning_logs/trv_grupool_l-1_gruio-768x512_krnls-1x1-1x1/epoch=29-val_loss=0.037-val_ap=0.356.ckpt', 
                                           grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)



poolkernels = [(1, 1), (3, 1), (3, 1)]
gru_in_out = [(768, 512), (512, 512)]
layers = (len(gru_in_out))
# 22 > 20
model_3 = TrackToTags.load_from_checkpoint('lightning_logs/trv_grupool_l-2_gruio-768x512-512x512_krnls-1x1-3x1-3x1/epoch=22-val_loss=0.034-val_ap=0.414.ckpt', 
                                           grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)

model_7 = TrackToTags.load_from_checkpoint('lightning_logs/trv_grupoolaccum_l-2_gruio-768x512-512x512_krnls-1x1-3x1-3x1/epoch=20-val_loss=0.046-val_ap=0.199.ckpt', 
                                           grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels)

# model_8 = TrackToTags.load_from_checkpoint('lightning_logs/grupoolworelu_l-2_gruio-768x512-512x512_krnls-1x1-3x1-3x1/epoch=25-val_loss=0.045-val_ap=0.239.ckpt', 
#                                            grulayers=layers, gru_in_out=gru_in_out, poolkernels=poolkernels, relu_on=False)


In [22]:
models_list = [model_1, model_3, model_7, model_8]
meta_factors_list = []   # num_models x [val_size x 256]
for model in models_list:
    track_idxs, preds = predict(model, dataloader_val_astest)
    meta_factors_list.append(preds)
val_true = np.array([y[2] for y in dataset_val])  # val_size x 256
meta_factors = np.concatenate(meta_factors_list, axis=1)  # val_size x 256*num_models
val_true.shape, meta_factors.shape

((10227, 256), (10227, 1024))

In [24]:
class MetaDataset(torch.utils.data.Dataset):
    def __init__(self, X, ground_true=None):
        self.ground_true = ground_true
        self.X = X

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        if self.ground_true is None:
            return self.X[idx]
        return self.X[idx], self.ground_true[idx]
    

def metacollate(batch):
    meta_factors = torch.FloatTensor(np.vstack([b[0] for b in batch])).to(global_device)
    ground_trues = torch.FloatTensor(np.vstack([b[1] for b in batch])).to(global_device)
    return meta_factors, ground_trues

def metacollate_test(batch):
    meta_factors = torch.FloatTensor(np.vstack(batch)).to(global_device)
    return meta_factors


metadataset_train = MetaDataset(meta_factors, ground_true=val_true)

metadataloader_train = DataLoader(metadataset_train, batch_size, shuffle=True, collate_fn=metacollate)
metadataloader_val = DataLoader(metadataset_train, batch_size, shuffle=False, collate_fn=metacollate)

In [63]:
class MetaModel(pl.LightningModule):
    def __init__(self, num_models, num_classes=256, hidden_dim=512):
        super().__init__()
        self.num_classes = num_classes
        input_dim = num_classes * num_models
        self.lin = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes),
        )
        # self.fc = nn.Linear(hidden_dim, num_classes)
        self.loss = nn.BCEWithLogitsLoss()
        self.metric = average_precision_score
        self._transit_val = {'preds': [], 'labels': []}
        
    def forward(self, x):
        outs = self.lin(x)
        # outs = self.fc(x)
        return outs

    def training_step(self, batch, batch_idx):
        meta_factors, ground_trues = batch
        pred_logits = self(meta_factors)
        loss = self.loss(pred_logits, ground_trues)
        self.log("train_loss", loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        meta_factors, ground_trues = batch
        pred_logits = self(meta_factors)
        loss = self.loss(pred_logits, ground_trues)
        self.log("val_loss", loss, prog_bar=True)
        
        pred_probs = torch.sigmoid(pred_logits)
        self._transit_val['labels'].append(np.array(ground_trues.int().cpu()))
        self._transit_val['preds'].append(np.array(pred_probs.cpu()))

    def on_validation_epoch_end(self):
        preds = np.vstack(self._transit_val['preds'])
        labels = np.vstack(self._transit_val['labels'])
        ap = self.metric(labels, preds)
        self.log('val_ap', ap, prog_bar=True)
        self._transit_val['labels'] = []
        self._transit_val['preds'] = []

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=3e-4)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5)
        
        return {"optimizer": optimizer,
                "lr_scheduler": {
                                "scheduler": scheduler,
                                "interval": "epoch",
                                "frequency": 10
                                },
               }

In [64]:
num_models = len(models_list)
checkpoint_callback = ModelCheckpoint(dirpath=f'lightning_logs/metamodel_{num_models}models/',
                                      filename='{epoch}-{val_loss:.3f}-{val_ap:.3f}', 
                                      save_top_k=-1, 
                                      monitor="val_ap", 
                                      every_n_epochs=1)
trainer = pl.Trainer(accelerator="gpu", devices=1, val_check_interval=1.0, 
                     max_epochs=70, log_every_n_steps=100,
                     callbacks=[checkpoint_callback])
metamodel = MetaModel(num_models=num_models)
trainer.fit(metamodel, metadataloader_train, metadataloader_val)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type              | Params
-------------------------------------------
0 | lin  | Sequential        | 657 K 
1 | loss | BCEWithLogitsLoss | 0     
-------------------------------------------
657 K     Trainable params
0         Non-trainable params
657 K     Total params
2.629     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch 69: 100%|██████████| 40/40 [00:01<00:00, 24.63it/s, v_num=295, train_loss=0.041, val_loss=0.0403, val_ap=0.308]  

`Trainer.fit` stopped: `max_epochs=70` reached.


Epoch 69: 100%|██████████| 40/40 [00:01<00:00, 23.81it/s, v_num=295, train_loss=0.041, val_loss=0.0403, val_ap=0.308]


In [82]:
metamodel = MetaModel.load_from_checkpoint(f'lightning_logs/metamodel_{num_models}models/epoch=38-val_loss=0.041-val_ap=0.299.ckpt', 
                                            num_models=num_models)

In [47]:
def metapredict(model, loader):
    model.to(global_device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for data in loader:
            pred_logits = model(data)
            pred_probs = torch.sigmoid(pred_logits)
            predictions.append(pred_probs.cpu().numpy())
    predictions = np.vstack(predictions)
    return predictions

In [29]:
meta_factors_list = []   # num_models x [test_size x 256]
for model in models_list:
    track_idxs, preds = predict(model.to(global_device), dataloader_test)
    meta_factors_list.append(preds)
meta_factors = np.concatenate(meta_factors_list, axis=1)  # test_size x 256*num_models
metadataset_test = MetaDataset(meta_factors)
metadataloader_test = DataLoader(metadataset_test, batch_size, shuffle=False, collate_fn=metacollate_test)

In [83]:
predictions = metapredict(metamodel, metadataloader_test)

In [50]:
track_idxs, predictions = predict(model_3.to(global_device), dataloader_test)

In [52]:
predictions_df = pd.DataFrame([
    {'track': track, 'prediction': ','.join([str(p) for p in probs])}
    for track, probs in zip(track_idxs, predictions)
])
predictions_df.to_csv('results/prediction_mod3.csv', index=False)

In [ ]:
for i, preds in enumerate(meta_factors_list):
    predictions_df = pd.DataFrame([
    {'track': track, 'prediction': ','.join([str(p) for p in probs])}
    for track, probs in zip(track_idxs, preds)
    ])
    predictions_df.to_csv(f'results/prediction_{i}.csv', index=False)
